In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical

import tensorflow as tf
import keras
import keras.backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.tensorflow_backend.set_session(sess)


LONGTERM = "/longterm/zifanw/Caltech/"
test_y = np.load(LONGTERM+'test_y.npy')
target_set_id = np.where(test_y == 55)[0]

test_x = np.load(LONGTERM+'test_x.npy')

In [ ]:
import explainer.QuantityInterests as Q
from explainer.Attribution import KerasAttr
from ruamel.yaml import YAML
from pathlib import Path
from densenet import densenet121_model
qoi = Q.ClassInterest(235)

path = Path('caltech_config.yaml')
yaml = YAML(typ='safe')
config = yaml.load(path)
img_rows, img_cols = config['img_row'], config['img_column']
img_channels = config['img_channel']
batch_size = config['batch_size']
nb_classes = config['nb_classes']

model = densenet121_model(img_rows=img_rows,
                          img_cols=img_cols,
                          color_type=img_channels,
                          num_classes=nb_classes,
                          weight_path_prefix="../")
print('=' * 50)
print('Model is created')
#     model.summary()

model.load_weights(config['weights_dir'] + config['pretrained'])




attr_creater = KerasAttr(model, qoi)

In [ ]:
X_test = np.load(config['X_test_path']).astype('float')
Y_test = np.load(config['Y_test_path'])

color_mean = [123.68, 116.779, 103.939]
for i in range(3):
    X_test[:, :, :, i] -= color_mean[i]

Y_test = to_categorical(Y_test, num_classes=nb_classes)
## Test the scores
scores = model.evaluate(X_test, Y_test, batch_size=batch_size)
print('Test loss : ', scores[0])
print('Test accuracy : ', scores[1])

In [ ]:
target_dataset = np.load(config['X_test_path']).astype('float')[target_set_id]
pred_scores = model.predict(target_dataset)
np.argmax(pred_scores, axis=-1)

In [ ]:
from explainer.Influence import KerasInflExp
import explainer as E
import explainer.Influence as Infl
import explainer.QuantityInterests as Q
import explainer.ExpertUnit as EU
import explainer.Compression as Cmpr
def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    indices = np.unravel_index(indices, ary.shape)
    x, y= indices[0], indices[1]
    xx = x.reshape((1, x.shape[0]))
    yy = y.reshape((1, y.shape[0]))
    result = [xx, yy]
    result = np.vstack(result)
    return result

infl_model = KerasInflExp(model, channel_first=False)

In [ ]:
target = target_dataset[1:2]

In [ ]:
saliency_map = attr_creater.saliency_map(target, batch_size=16, mul_with_input=True)
integrated_grad = attr_creater.integrated_grad(target, batch_size=16, mul_with_input=True)
smooth_grad = attr_creater.smooth_grad(target, batch_size=16, mul_with_input=True)
grad_cam = attr_creater.gradcam(target, layer_name="concatenate_58", batch_size=16, channel_first=False)

In [ ]:
infl_model.set_doi_type('internal')
qoi = Q.ClassInterest(55)
raw_infl = infl_model.internal_infl(
    X_test,
    from_layer='fc6',  ## Use the pre-Softmax scores
    wrt_layer="concatenate_55",
    interest_mask=qoi)
expert = EU.Expert("Neuron")
top_idx = np.arange(10)
expert(raw_infl)
tops = expert.get_tops()
inter_unit_wts = expert.get_inter_unit_wts(top_idx)
multi_neuron_vis = infl_model.multi_unit_visualizaiton(target, 
                                                       "concatenate_55", 
                                                       tops[:10], 
                                                       inter_unit_wts=inter_unit_wts, 
                                                       infl_as_wts=False,
                                                       multiply_with_input=True)
raw_infl = infl_model.internal_infl(
    X_test,
    from_layer='fc6',  ## Use the pre-Softmax scores
    wrt_layer="relu5_11_x1",
    interest_mask=qoi)
channel_expert = EU.Expert("Channel")
channel_expert(raw_infl, heuristic='max', channel_first=False)
channel_tops = channel_expert.get_tops()
inter_unit_wts = channel_expert.get_inter_unit_wts(np.array([0, 1, 2]))
multi_channel_vis = infl_model.multi_unit_visualizaiton(target, 
                                                         'relu5_11_x1', 
                                                         channel_tops[:3], 
                                                         inter_unit_wts=inter_unit_wts, 
                                                         infl_as_wts=True, 
                                                         interest_mask=qoi,
                                                         multiply_with_input=True)

In [ ]:
stops = expert.get_tops()[[0, 1, 2]]

In [ ]:
stops.T[0], stops.T[1], stops.T[2]

In [ ]:
infl_model.set_doi_type('integrated')
multi_neuron_vis_ig = infl_model.multi_unit_visualizaiton(target, 
                                                       "concatenate_55", 
                                                       tops[:10], 
                                                       inter_unit_wts=inter_unit_wts, 
                                                       infl_as_wts=False,
                                                       multiply_with_input=True)



In [ ]:
np.save("multi_neuron_vis_ig.npy", multi_neuron_vis_ig)

In [ ]:
multi_neuron_vis_ig = np.load("multi_neuron_vis_ig.npy")

In [ ]:
infl_model.set_doi_type('integrated')
multi_channel_vis_ig = infl_model.multi_unit_visualizaiton(target, 
                                                         'relu5_11_x1', 
                                                         channel_tops[:3], 
                                                         inter_unit_wts=inter_unit_wts, 
                                                         infl_as_wts=True, 
                                                         interest_mask=qoi,
                                                         multiply_with_input=True)

In [ ]:
infl_model.set_doi_type('smooth')
qoi = Q.ClassInterest(55)
raw_infl = infl_model.internal_infl(
    X_test,
    from_layer='fc6',  ## Use the pre-Softmax scores
    wrt_layer="concatenate_55",
    interest_mask=qoi)
expert = EU.Expert("Neuron")
top_idx = np.arange(10)
expert(raw_infl)
tops = expert.get_tops()
inter_unit_wts = expert.get_inter_unit_wts(top_idx)
multi_neuron_vis_sm = infl_model.multi_unit_visualizaiton(target, 
                                                       "concatenate_55", 
                                                       tops[:10], 
                                                       inter_unit_wts=inter_unit_wts, 
                                                       infl_as_wts=False,
                                                       multiply_with_input=True)



In [ ]:
infl_model.set_doi_type('smooth')
multi_channel_vis_sm = infl_model.multi_unit_visualizaiton(target, 
                                                         'relu5_11_x1', 
                                                         channel_tops[:3], 
                                                         inter_unit_wts=inter_unit_wts, 
                                                         infl_as_wts=True, 
                                                         interest_mask=qoi,
                                                         multiply_with_input=True)

In [ ]:
from explainer.Visualization import binary_mask, cv2_heatmap_mask

threhsold = 0.2
blur = 3

v_saliency_map = binary_mask(target, saliency_map, threshold=threhsold, background=0.3, blur=blur)
v_integrated_grad = binary_mask(target, integrated_grad, threshold=threhsold, background=0.3, blur=blur)
v_smooth_grad = binary_mask(target, smooth_grad, threshold=threhsold, background=0.3, blur=blur)

v_infl_neuron = binary_mask(target, multi_neuron_vis, threshold=threhsold, background=0.3, blur=blur)
v_infl_neuron_ig = binary_mask(target, multi_neuron_vis_ig, threshold=threhsold, background=0.3, blur=blur)
v_infl_neuron_sm = binary_mask(target, multi_neuron_vis_sm, threshold=threhsold, background=0.3, blur=blur)



def show_expl():
    fig, ax = plt.subplots(2, 3, figsize=(15, 10))
    ax[0, 0].imshow(v_saliency_map[0])
    ax[0, 1].imshow(v_integrated_grad[0])
    ax[0, 2].imshow(v_smooth_grad[0])
    ax[1, 0].imshow(v_infl_neuron[0])
    ax[1, 1].imshow(v_infl_neuron_ig[0])
    ax[1, 2].imshow(v_infl_neuron_sm[0])
    
    ax[0, 0].set_title('Saliency Map')
    ax[0, 1].set_title('IG')
    ax[0, 2].set_title('SM')
    ax[1, 0].set_title('Neuron Infl')
    ax[1, 1].set_title('Neuron Infl IG')
    ax[1, 2].set_title('Neuron Infl SM')

    
    plt.show()

show_expl()

In [ ]:
from explainer.Visualization import point_cloud

In [ ]:
attributions = [saliency_map[0].mean(axis=-1), 
                integrated_grad[0].mean(axis=-1), 
                smooth_grad[0].mean(axis=-1), 
                grad_cam[0].mean(axis=-1),
                point_cloud(multi_neuron_vis)[0], 
                point_cloud(multi_neuron_vis_ig)[0],
                point_cloud(multi_neuron_vis_sm)[0],
                np.random.randn(224, 224)]

scores = []
for method in attributions:
    pos_attr_id = largest_indices(method, 1000)
    result = []
    for i in range(pos_attr_id.shape[1]):
        img = target[0].copy()
        img[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 0
        result.append(img[None, :])

    removed = np.vstack(result)
    pre_softmax_scores = infl_model.get_activation(removed, "fc6")
    scores.append(pre_softmax_scores[:, 55])

In [ ]:
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

layout = go.Layout(
    title='Remove pixels with high attribution',
    xaxis=dict(
        title='The number of removed pixels',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Pre-Softmax Score',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(layout=layout)

# Add traces
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=scores[0],
                    mode='lines',
                    name='SaliencyMap'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=scores[1],
                    mode='lines',
                    name='IG'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=scores[2],
                    mode='lines',
                    name='SM'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=scores[3],
                    mode='lines',
                    name='GradCAM'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=scores[4],
                    mode='lines',
                    name='N_Infl'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=scores[5],
                    mode='lines',
                    name='N_Infl_ig'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=scores[6],
                    mode='lines',
                    name='N_Infl_sm'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=scores[7],
                    mode='lines',
                    name='Random'))

fig.show()